LOAD MODELS AND DEPENDENCIES

In [33]:
#Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
import os

# List files in your model directory
print(os.listdir('/content/drive/MyDrive/saved_models'))

['C4_values.npy', 'C6_values.npy', 'C5_values.npy', 'C3_values.npy', 'C2_values.npy', 'facial_model.h5', 'survey_model.h5', 'audio_model.h5', 'survey_scaler.pkl', 'survey_encoder.pkl']


In [35]:
#Loads pre-trained models from model directory
import tensorflow as tf
facial_model = tf.keras.models.load_model("/content/drive/MyDrive/saved_models/facial_model.h5") #Facial Prediction
audio_model = tf.keras.models.load_model("/content/drive/MyDrive/saved_models/audio_model.h5") #Audio Prediction

FACIAL

In [36]:
import cv2
import numpy as np
import tensorflow as tf

#Function that takes an image path and loads model
def predict_facial(photo_path, model):

    """
    photo_path: path to the photo file
    model: loaded keras facial model

    Returns:
        label: 'Stressed' or 'Not Stressed'
        confidence: float, confidence for the predicted label
    """

    # Load and preprocess image
    #Reads the image from photo path, converts it to grayscale, resized and normalizes it
    img = cv2.imread(photo_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, (48, 48))
    normalized = resized / 255.0
    input_img = normalized.reshape(1, 48, 48, 1)

    # Predict
    prediction = model.predict(input_img, verbose=0)
    class_id = np.argmax(prediction)
    confidence = float(prediction[0][class_id])
    label = "Stressed" if class_id == 1 else "Not Stressed"
    return label, confidence

SURVEY

In [37]:
import tensorflow as tf
import joblib
import numpy as np
import pandas as pd

# Load your trained model
model = tf.keras.models.load_model('/content/drive/MyDrive/saved_models/survey_model.h5')

# Load the fitted encoder and scaler
encoder = joblib.load('/content/drive/MyDrive/saved_models/survey_encoder.pkl')
scaler = joblib.load('/content/drive/MyDrive/saved_models/survey_scaler.pkl')

In [38]:
# Load unique values for each question (for Gradio radio buttons)
features = ['C2', 'C3', 'C4', 'C5', 'C6']
choices = []
for col in features:
    choices.append(sorted(np.load(f"/content/drive/MyDrive/saved_models/{col}_values.npy", allow_pickle=True)))

#Function to predict stressed/not stressed based on survey answered by the user.
def predict_survey(activity, companion, pressure, tired, energetic):
    user_input = pd.DataFrame([{
        'C2': activity,
        'C3': companion,
        'C4': pressure,
        'C5': tired,
        'C6': energetic
    }])
    encoded = encoder.transform(user_input[features])
    scaled = scaler.transform(encoded)
    prediction = model.predict(scaled)[0][0]
    label = "Stressed" if prediction >= 0.5 else "Not Stressed"
    confidence = round(float(prediction if prediction >= 0.5 else 1 - prediction), 2)
    return label, confidence


AUDIO

In [39]:
import tensorflow as tf
import numpy as np
import librosa

#Audio model already loaded globally but can be loaded again.
#audio_model = tf.keras.models.load_model('/content/drive/MyDrive/saved_models/audio_model.h5')

#Function to convert the audio waveform into machine-readable 2d feature map (MFCCs)
def extract_features(audio, sr=22050):
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=38)

    # Pad or crop to 98 time steps
    if mfccs.shape[1] < 98:
        pad_width = 98 - mfccs.shape[1]
        mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
    else:
        mfccs = mfccs[:, :98]

    mfccs = np.expand_dims(mfccs, axis=-1)  # (38, 98, 1)
    return mfccs

#Function to predict stress based on the MFCC made by previous function
def predict_audio(audio_file, model=audio_model):
    try:
        y, sr = librosa.load(audio_file, sr=22050)
        features = extract_features(y, sr)
        features = features.reshape(1, 38, 98, 1)  # Input shape for CNN

        prediction = model.predict(features, verbose=0)[0][0]  # Binary classification

        label = "Stressed" if prediction >= 0.5 else "Not Stressed"
        confidence = prediction if prediction >= 0.5 else 1 - prediction

        return label, confidence

    except Exception as e:
        print(f"❌ Audio Prediction Error: {str(e)}")
        return "Error", 0.0


FUSION

In [40]:
import numpy as np

#Function to get the fused(weighted) average confidence of the prediction
def get_stress_confidence(label, confidence):
    if label.lower() == 'stressed':
        return float(confidence)
    else:
        return 1.0 - float(confidence)

def agreement_fusion(confidences):
    M = len(confidences)
    agree_scores = []
    for i in range(M):
        total_agree = 0
        for j in range(M):
            if i != j:
                total_agree += (1 - abs(confidences[i] - confidences[j]))
        agree_i = total_agree / (M - 1)
        agree_scores.append(agree_i)
    agree_scores = np.array(agree_scores)
    fused = np.sum(agree_scores * confidences) / np.sum(agree_scores)
    return fused

In [41]:
import gradio as gr
def fused_stress_prediction(audio_file, photo_path,
                            activity, companion, pressure, tired, energetic):

    try:
        print("📥 Inputs Received:")
        print("Audio:", audio_file)
        print("Image:", photo_path)
        print("Survey:", activity, companion, pressure, tired, energetic)

        # 🔍 Predict from individual modalities
        label_audio, conf_audio = predict_audio(audio_file, audio_model)
        label_facial, conf_facial = predict_facial(photo_path, facial_model)
        label_survey, conf_survey = predict_survey(activity, companion, pressure, tired, energetic)

        # 🔗 Fuse predictions into a single score
        confidences = [
            get_stress_confidence(label_audio, conf_audio),
            get_stress_confidence(label_facial, conf_facial),
            get_stress_confidence(label_survey, conf_survey)
        ]
        fused_score = round(agreement_fusion(confidences), 2)
        print(f"🔁 Fused Stress Score: {fused_score:.2f}")

        # 🧠 Final label
        label = "🧠 Stressed" if fused_score >= 0.5 else "🙂 Not Stressed"

        return f"{label} (Fused Score: {fused_score:.2f})"

    except Exception as e:
        return f"⚠️ Error: {str(e)}"

choices = [
    ["Working", "Studying", "Relaxing", "Exercising", "Other"],
    ["Alone", "Friends", "Family", "Colleagues", "Strangers"],
    ["Yes", "No", "Maybe"],
    ["Yes", "No"],
    ["Very Energetic", "Somewhat Energetic", "Neutral", "Tired"]
]
# Gradio interface for increase ease of use
iface = gr.Interface(
    fn=fused_stress_prediction,
    inputs=[
        gr.Audio(type="filepath", label="Voice"),
        gr.Image(type="filepath", label="Photo"),
        gr.Radio(choices=choices[0], label="What activity are you currently engaged in?"),
        gr.Radio(choices=choices[1], label="Who are you currently with?"),
        gr.Radio(choices=choices[2], label="Do you feel under pressure right now?"),
        gr.Radio(choices=choices[3], label="Do you feel tired?"),
        gr.Radio(choices=choices[4], label="Do you feel energetic right now?")
    ],
    outputs="text"
)

iface.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9cc700a847d376e5e6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
